In [ ]:
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle datasets download -d grassknoted/asl-alphabet

 99% 1.01G/1.03G [00:09<00:00, 82.9MB/s]
100% 1.03G/1.03G [00:09<00:00, 115MB/s] 


In [ ]:
!unzip asl-alphabet.zip -d asl-alphabet

In [ ]:
# Load Data
import os
import cv2
import numpy as np

# Data Visualisation
import matplotlib.pyplot as plt

# Model Training
from tensorflow.keras import utils
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16

# Warning
import warnings
warnings.filterwarnings("ignore")

# Main
import os
import glob
import cv2
import numpy as np
import pandas as pd
import gc
import string
import time
import random
from PIL import Image
from tqdm import tqdm
tqdm.pandas()

# Visualization
import matplotlib
import matplotlib.pyplot as plt




from sklearn.manifold import TSNE

# Model
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from keras.models import load_model, Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import classification_report

In [ ]:
# Configuration
class CFG:
    # Set the batch size for training
    batch_size = 128
    # Set the height and width of input images
    img_height = 32
    img_width = 32
    epochs = 10
    num_classes = 29
    # Define the number of color channels in input images
    img_channels = 3
# Define a function to set random seeds for reproducibility
def seed_everything(seed: int):
    random.seed(seed)
    # Set the environment variable for Python hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [ ]:
# Labels
TRAIN_PATH = "/content/asl-alphabet/asl_alphabet_train/asl_alphabet_train"
labels = []
# Generate a list of uppercase letters in the English alphabet
alphabet = list(string.ascii_uppercase)
labels.extend(alphabet)
# Add special labels for 'delete', 'nothing', and 'space' gestures
labels.extend(["del", "nothing", "space"])
print(labels)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


In [ ]:
# Create Metadata
list_path = []
list_labels = []
for label in labels:
    # Create a path pattern to match all image files for the current label
    label_path = os.path.join(TRAIN_PATH, label, "*")
    # Use glob to retrieve a list of image file paths that match the pattern
    image_files = glob.glob(label_path)
    sign_label = [label] * len(image_files)
    list_path.extend(image_files)
    list_labels.extend(sign_label)
metadata = pd.DataFrame({
    "image_path": list_path,
    "label": list_labels
})
metadata

,image_path,label
0,/content/asl-alphabet/asl_alphabet_train/asl_a...,A
1,/content/asl-alphabet/asl_alphabet_train/asl_a...,A
2,/content/asl-alphabet/asl_alphabet_train/asl_a...,A
3,/content/asl-alphabet/asl_alphabet_train/asl_a...,A
4,/content/asl-alphabet/asl_alphabet_train/asl_a...,A
...,...,...
86995,/content/asl-alphabet/asl_alphabet_train/asl_a...,space
86996,/content/asl-alphabet/asl_alphabet_train/asl_a...,space
86997,/content/asl-alphabet/asl_alphabet_train/asl_a...,space
86998,/content/asl-alphabet/asl_alphabet_train/asl_a...,space


In [ ]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    metadata['image_path'],
    metadata['label'],
    test_size=0.2,
    random_state=2253,
    shuffle=True,
    stratify=metadata['label']
)

# Create a DataFrame for the training set test set
data_train = pd.DataFrame({
    'image_path': X_train,
    'label': y_train
})

data_test = pd.DataFrame({
    'image_path': X_test,
    'label': y_test
})

# Split the training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    data_train['image_path'],
    data_train['label'],
    test_size=0.2/0.7,  # Assuming you want 20% for validation out of the training set
    random_state=2253,
    shuffle=True,
    stratify=data_train['label']
)

# Create a DataFrame for the validation set
data_val = pd.DataFrame({
    'image_path': X_val,
    'label': y_val
})

In [ ]:
def data_augmentation():
    datagen = ImageDataGenerator(
        rescale=1/255.,
        # Add other augmentation parameters as needed
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    train_generator = datagen.flow_from_dataframe(
        data_train,
        directory='./',
        x_col='image_path',
        y_col='label',
        class_mode='categorical',
        batch_size=CFG.batch_size,
        target_size=(CFG.img_height, CFG.img_width)
    )

    validation_generator = datagen.flow_from_dataframe(
        data_val,
        directory='./',
        x_col='image_path',
        y_col='label',
        class_mode='categorical',
        batch_size=CFG.batch_size,
        target_size=(CFG.img_height, CFG.img_width)
    )

    test_generator = datagen.flow_from_dataframe(
        data_test,  # Assuming you have a DataFrame for test data
        directory='./',
        x_col='image_path',
        y_col='label',
        class_mode='categorical',
        batch_size=CFG.batch_size,
        target_size=(CFG.img_height, CFG.img_width),
        shuffle=False  # Set to False for test data
    )

    return train_generator, validation_generator, test_generator

# Seed for reproducibility
seed_everything(2253)

# Get the generators
train_generator, validation_generator, test_generator = data_augmentation()

Found 69600 validated image filenames belonging to 29 classes.
Found 19886 validated image filenames belonging to 29 classes.
Found 17400 validated image filenames belonging to 29 classes.


In [ ]:
# Define input shape
input_shape = (32, 32, 3)

# Load the VGG16 model without the top (classification) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# Add your custom classification layers on top of the base model
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)  # You can adjust the number of units as needed
predictions = Dense(29, activation='softmax')(x)  # num_classes is the number of classes in your dataset

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Summarize the model architecture
model.summary()


58889256/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                              

In [ ]:
# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Create a ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath='/content/sample_data/best_model_weights.h5',
    monitor='val_accuracy',  # Monitor validation accuracy for saving the best model
    save_best_only=True,
    mode='max',
    verbose=1
)
# Train the model using the fit method
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // CFG.batch_size,  # Number of steps per epoch
    epochs=CFG.epochs,  # Number of training epochs
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // CFG.batch_size,  # Number of validation steps
    callbacks=[checkpoint_callback],
    shuffle=True,
    verbose=1
)


Epoch 1/10
543/543 [==============================] - ETA: 0s - loss: 2.6236 - accuracy: 0.1762
Epoch 1: val_accuracy improved from -inf to 0.41754, saving model to /content/sample_data/best_model_weights.h5
543/543 [==============================] - 163s 273ms/step - loss: 2.6236 - accuracy: 0.1762 - val_loss: 1.6868 - val_accuracy: 0.4175
Epoch 2/10
543/543 [==============================] - ETA: 0s - loss: 1.0716 - accuracy: 0.6257
Epoch 2: val_accuracy improved from 0.41754 to 0.78942, saving model to /content/sample_data/best_model_weights.h5
543/543 [==============================] - 130s 239ms/step - loss: 1.0716 - accuracy: 0.6257 - val_loss: 0.6494 - val_accuracy: 0.7894
Epoch 3/10
543/543 [==============================] - ETA: 0s - loss: 0.5376 - accuracy: 0.8276
Epoch 3: val_accuracy improved from 0.78942 to 0.84824, saving model to /content/sample_data/best_model_weights.h5
543/543 [==============================] - 144s 265ms/step - loss: 0.5376 - accuracy: 0.8276 - val_l

In [ ]:
scores = model.evaluate(test_generator)
print("%s: %2f%%" % ("Evaluate Test Accuracy", scores[1]*100))

136/136 [==============================] - 25s 182ms/step - loss: 0.1469 - accuracy: 0.9626
Evaluate Test Accuracy: 96.264368%


In [ ]:
# Confusion Matrix
fine_tuned_model = load_model("/content/sample_data/best_model_weights.h5")
predictions = fine_tuned_model.predict(test_generator)

# Get the true labels from the generator
true_labels = test_generator.classes

# Compute the confusion matrix using tf.math.confusion_matrix
confusion_matrix = tf.math.confusion_matrix(
    labels = true_labels,
    predictions = predictions.argmax(axis=1),
    num_classes = 29
)
print(confusion_matrix)

136/136 [==============================] - 25s 178ms/step
tf.Tensor(
[[578   1   0   0   2   0   0   0   0   0   0   0   5   1   0   0   0   0
    7   1   1   0   0   1   0   3   0   0   0]
 [  0 585   0   0   1   1   0   0   0   0   1   0   0   0   1   0   0   5
    0   0   5   0   0   0   0   0   0   1   0]
 [  0   1 585   0   0   0   0   3   0   0   0   0   0   0   3   0   1   0
    1   1   0   0   0   0   0   0   2   0   3]
 [  0   6   0 584   0   2   0   0   1   0   0   0   0   0   5   0   0   1
    0   0   0   1   0   0   0   0   0   0   0]
 [ 12   6   0   0 565   1   0   0   5   0   0   0   2   0   1   0   0   0
    7   0   0   1   0   0   0   0   0   0   0]
 [  0   2   0   3   1 590   0   0   0   0   0   0   1   0   1   0   0   0
    1   0   0   0   0   0   1   0   0   0   0]
 [  3   0   0   0   0   0 572   5   2   6   0   0   0   0   0   5   0   0
    2   0   0   2   0   0   1   1   0   0   1]
 [  0   0   0   0   0   0   5 589   0   1   0   0   0   0   1   0   0   0
    0   0 

In [ ]:
#Classification report
predictions = model.predict(test_generator)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = test_generator.classes
report = classification_report(true_labels, predicted_labels, target_names=labels)
print(report)

136/136 [==============================] - 25s 181ms/step
              precision    recall  f1-score   support

           A       0.95      0.95      0.95       600
           B       0.94      0.96      0.95       600
           C       1.00      0.97      0.98       600
           D       0.99      0.98      0.98       600
           E       0.98      0.94      0.96       600
           F       0.99      0.97      0.98       600
           G       0.99      0.95      0.97       600
           H       0.97      0.98      0.97       600
           I       0.97      0.96      0.96       600
           J       0.97      0.97      0.97       600
           K       0.94      0.91      0.93       600
           L       1.00      0.96      0.98       600
           M       0.93      0.93      0.93       600
           N       0.94      0.95      0.94       600
           O       0.97      0.99      0.98       600
           P       0.98      0.98      0.98       600
           Q       0.99

In [ ]:
# Load the saved model
model = tf.keras.models.load_model('/content/sample_data/best_model_weights.h5')

In [ ]:
# Testing with an image
image_path = '/content/asl-alphabet/asl_alphabet_train/asl_alphabet_train/L/L100.jpg'
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img,(32,32))

img = tf.keras.applications.mobilenet_v2.preprocess_input(img)

# Predict the class of the image
predictions = model.predict(np.array([img]))

# Get the class with the highest probability
predicted_class = labels[np.argmax(predictions)]

print(f"The image is predicted to belong to class: {predicted_class}")

1/1 [==============================] - 0s 36ms/step
The image is predicted to belong to class: L
